In [313]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [314]:
# settings
np.random.seed(2020)
name = "Dblp"
d = 128
come_model_type = "GMM"
ks = [2, 5, 10, 20]

In [315]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [316]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/minico

k = 2
k = 5
k = 10
k = 20


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.305006,-0.466880,-0.267545,0.769969,0.285500,0.145671,0.145668,0.589570,0.057389,0.295072,...,0.022924,-0.109912,-0.527391,-0.116635,-0.446284,0.117288,-0.009859,-0.156843,1,0
1,0.274115,-0.487077,-0.116241,0.739823,0.195494,0.060871,0.084048,0.557284,-0.170716,0.435309,...,-0.059694,-0.219559,-0.513448,-0.049340,-0.475041,0.052064,0.117735,-0.032222,1,2
2,0.417771,-0.439148,-0.307630,0.787992,0.405380,0.128854,0.119469,0.588898,0.191887,0.283263,...,0.006722,-0.004066,-0.469753,-0.177917,-0.386277,0.056969,0.023117,-0.182263,1,2
3,0.306671,-0.452776,-0.340008,0.667185,0.238152,0.025103,0.174575,0.546116,0.084083,0.320654,...,-0.080180,-0.132030,-0.433514,-0.166257,-0.302751,0.087579,0.046053,-0.229579,1,0
4,0.218821,-0.380524,-0.374861,0.763850,0.261986,0.098802,0.026748,0.559295,0.217035,0.293235,...,0.038992,-0.054769,-0.509383,-0.214571,-0.346377,0.035100,-0.079765,-0.218875,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.233857,-0.375045,-0.195243,0.585578,0.377580,0.057074,0.045089,0.492758,0.113148,0.298501,...,-0.154959,-0.091873,-0.352352,-0.144252,-0.037096,0.055319,0.095775,-0.173232,1,0
13180,-0.128917,-0.556151,-0.205546,-0.030483,0.330267,0.210279,0.195406,0.034497,0.400683,0.501986,...,-0.314047,0.298250,-0.008569,0.241477,-0.022174,-0.508296,0.082054,0.339728,0,3
13181,-0.181398,-0.531019,-0.020660,0.022836,0.288119,0.118601,0.141081,0.019796,0.511212,0.564311,...,-0.129990,0.343729,-0.115812,0.100923,-0.177653,-0.494129,0.069173,0.356277,0,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.051781,-0.302547,0.048093,0.296949,0.298413,-0.073264,0.185268,0.308804,0.186108,0.388025,...,-0.167927,0.258325,-0.256381,0.007310,-0.071415,-0.042185,0.105839,0.172020,0.390246,2.190382
std,0.167067,0.202704,0.173806,0.237132,0.161112,0.184697,0.166071,0.181970,0.202842,0.152158,...,0.165648,0.178690,0.209837,0.135285,0.182414,0.155744,0.183775,0.225410,0.487824,1.315339
min,-0.738680,-0.984638,-0.549125,-0.502673,-0.317483,-0.719233,-0.405115,-0.421883,-0.457067,-0.285070,...,-0.832682,-0.348916,-0.878605,-0.662500,-0.708759,-0.751996,-0.541037,-0.464598,0.000000,0.000000
25%,-0.166453,-0.457009,-0.067273,0.117994,0.195739,-0.211840,0.073862,0.186877,0.040458,0.289796,...,-0.280665,0.133314,-0.419754,-0.076076,-0.197066,-0.142536,-0.017640,-0.002352,0.000000,1.000000
50%,-0.048913,-0.287717,0.043891,0.295192,0.302898,-0.076033,0.177041,0.308882,0.173247,0.393894,...,-0.158188,0.253377,-0.263822,0.009260,-0.063929,-0.041156,0.088836,0.181500,0.000000,3.000000
75%,0.064279,-0.150278,0.160756,0.484957,0.405333,0.067229,0.292778,0.428846,0.313818,0.489991,...,-0.049476,0.369090,-0.088316,0.090387,0.059805,0.058771,0.213611,0.341792,1.000000,3.000000
max,0.573231,0.312830,0.729769,0.985580,0.866095,0.530542,0.880256,0.977802,0.926466,0.978306,...,0.388878,1.005381,0.456588,0.543176,0.444104,0.530632,0.853895,0.845029,1.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.337873,-0.266789,-0.329776,0.567795,0.450980,0.215529,0.052986,0.404575,0.172645,0.233489,...,0.072292,-0.381494,-0.569689,-0.276140,-0.554999,0.046562,0.013247,-0.322961,1,0
1,0.298539,-0.215489,-0.122721,0.435305,0.372944,0.170758,0.011999,0.497292,0.042238,0.476271,...,-0.081790,-0.461628,-0.629205,-0.066755,-0.621623,-0.045337,0.262442,-0.200994,1,2
2,0.434990,-0.162728,-0.427830,0.554695,0.505497,0.257215,0.005215,0.411108,0.253336,0.216690,...,0.013009,-0.220384,-0.587793,-0.355655,-0.518804,0.010547,0.061988,-0.298181,1,2
3,0.371709,-0.187498,-0.304633,0.448847,0.348340,0.166682,0.078679,0.418406,0.245663,0.281862,...,-0.072003,-0.284714,-0.431775,-0.324367,-0.421556,-0.002468,0.041918,-0.312724,1,0
4,0.153169,-0.159715,-0.291998,0.493514,0.408746,0.214735,-0.123262,0.373804,0.286365,0.186570,...,0.108366,-0.217525,-0.571317,-0.419269,-0.446043,-0.080026,-0.001058,-0.280553,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.275016,-0.283070,-0.246501,0.375144,0.365492,0.030761,-0.037413,0.570026,0.238857,0.371782,...,-0.211221,-0.306782,-0.396477,-0.368964,-0.272261,-0.041332,0.295788,-0.166466,1,0
13180,-0.160261,-0.510791,-0.266001,0.040446,0.488711,0.160373,-0.006505,0.058254,0.456568,0.557354,...,-0.317196,0.299070,-0.036974,0.201134,0.074295,-0.503660,0.042052,0.427558,2,3
13181,-0.146448,-0.468140,-0.219674,0.019405,0.480868,0.029808,0.059457,-0.076149,0.538575,0.580266,...,-0.125511,0.270827,-0.053874,0.061566,-0.070960,-0.488494,0.023676,0.480988,2,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.053282,-0.233463,0.010987,0.229220,0.307893,-0.078662,0.109107,0.251692,0.152121,0.407082,...,-0.101677,0.152634,-0.214756,-0.043944,-0.044679,-0.013392,0.117200,0.106993,2.205097,2.190382
std,0.181886,0.208281,0.207804,0.199277,0.182113,0.229345,0.194830,0.193287,0.199168,0.179802,...,0.179247,0.194063,0.218864,0.170414,0.197512,0.183881,0.212970,0.244757,1.327660,1.315339
min,-0.787460,-0.977597,-0.687779,-0.439214,-0.346363,-0.776780,-0.558109,-0.446612,-0.451948,-0.378929,...,-0.918916,-0.568448,-1.018059,-0.820416,-0.723286,-0.765348,-0.446309,-0.601428,0.000000,0.000000
25%,-0.167973,-0.386186,-0.130100,0.090572,0.186888,-0.249399,-0.026755,0.124395,0.004065,0.287845,...,-0.216879,0.023837,-0.367845,-0.143557,-0.187950,-0.132204,-0.033471,-0.075737,1.000000,1.000000
50%,-0.052691,-0.231325,-0.002082,0.232109,0.309923,-0.078852,0.099826,0.247003,0.151768,0.407598,...,-0.090312,0.160053,-0.219113,-0.033839,-0.059995,-0.013658,0.096315,0.092946,2.000000,3.000000
75%,0.069406,-0.086332,0.134493,0.363762,0.433545,0.082539,0.243270,0.374408,0.292640,0.535767,...,0.019946,0.284622,-0.049546,0.058915,0.096505,0.107109,0.238979,0.281623,3.000000,3.000000
max,0.586813,0.549692,0.824517,0.880733,0.963684,0.678440,0.852517,1.020408,0.764161,1.022475,...,0.462731,0.781859,0.525975,0.683949,0.622039,0.663546,0.932250,0.897228,4.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.282678,-0.378229,-0.462555,0.654327,0.294055,0.064782,0.004454,0.348092,0.258756,0.258577,...,0.125976,-0.268933,-0.579994,-0.209670,-0.551970,0.195866,0.013276,-0.278391,2,0
1,0.275363,-0.300289,-0.179202,0.597150,0.218187,0.115032,0.016029,0.422201,0.081029,0.458462,...,0.025035,-0.405887,-0.597514,-0.042855,-0.595379,0.021309,0.216610,-0.221665,2,2
2,0.386403,-0.291904,-0.514043,0.665214,0.317156,0.145693,-0.025238,0.322712,0.316961,0.173248,...,0.105122,-0.146849,-0.603984,-0.279419,-0.566396,0.083877,0.069771,-0.262865,2,2
3,0.307715,-0.339279,-0.483907,0.430061,0.291970,0.042118,0.074469,0.296509,0.333280,0.153258,...,0.059130,-0.209256,-0.553800,-0.218648,-0.457825,0.081485,0.013008,-0.365603,2,0
4,0.186632,-0.313281,-0.473726,0.577534,0.267270,0.038498,-0.171268,0.328380,0.329067,0.218318,...,0.086623,-0.157981,-0.569316,-0.211192,-0.422620,0.008862,0.055889,-0.414729,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.309975,-0.315715,-0.151497,0.372034,0.457501,-0.024270,-0.064662,0.415138,0.267264,0.265314,...,-0.151939,-0.346558,-0.347131,-0.292222,-0.190496,-0.031664,0.238495,-0.262648,2,0
13180,-0.171464,-0.423908,-0.339180,0.018491,0.420459,0.204863,-0.114288,0.127939,0.436755,0.476099,...,-0.206340,0.285466,-0.067773,0.282150,0.103847,-0.605473,0.012163,0.243452,6,3
13181,-0.057865,-0.374082,-0.347110,-0.055801,0.360907,0.182993,-0.047708,-0.025280,0.459628,0.484556,...,-0.068967,0.241956,-0.065416,0.208287,0.093221,-0.597375,-0.081279,0.248965,6,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.031473,-0.229142,0.000674,0.239840,0.279514,-0.087551,0.093213,0.255676,0.155655,0.392075,...,-0.089674,0.157132,-0.203209,-0.045942,-0.051092,0.001313,0.091826,0.103152,4.010771,2.190382
std,0.184683,0.215433,0.211821,0.214869,0.189401,0.224903,0.205255,0.203022,0.204488,0.187339,...,0.180333,0.196387,0.213932,0.177722,0.219119,0.198182,0.216878,0.226211,2.655645,1.315339
min,-0.810310,-0.957557,-0.714972,-0.478251,-0.421070,-0.815403,-0.572082,-0.453781,-0.433116,-0.377536,...,-0.831665,-0.594042,-1.035168,-0.872346,-0.709719,-0.841139,-0.548250,-0.605248,0.000000,0.000000
25%,-0.149894,-0.380369,-0.143686,0.088948,0.153182,-0.260468,-0.044379,0.123445,0.004684,0.270990,...,-0.204234,0.021064,-0.345474,-0.155437,-0.209171,-0.123321,-0.063966,-0.065190,2.000000,1.000000
50%,-0.029676,-0.228651,-0.001633,0.240505,0.284878,-0.093708,0.086212,0.244896,0.150637,0.395267,...,-0.080432,0.159781,-0.196461,-0.037214,-0.061635,0.002204,0.070204,0.101945,4.000000,3.000000
75%,0.091452,-0.076560,0.137674,0.389890,0.407966,0.072832,0.236066,0.385920,0.293981,0.524569,...,0.030529,0.292161,-0.055561,0.063835,0.105815,0.129481,0.221432,0.261150,6.000000,3.000000
max,0.554368,0.517005,0.781441,0.943553,0.961111,0.680394,0.808649,0.974774,0.835659,1.026149,...,0.512941,0.834830,0.557077,0.716513,0.705960,0.747884,0.928530,0.859511,9.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.240801,-0.447054,-0.378992,0.714646,0.405573,0.150098,0.015428,0.463993,0.197108,0.268619,...,0.067953,-0.450027,-0.383891,-0.113837,-0.612211,0.196843,0.049171,-0.281966,18,0
1,0.394008,-0.344323,-0.145407,0.546544,0.199697,0.096610,0.113022,0.618293,-0.007435,0.508423,...,-0.014885,-0.516593,-0.525129,-0.017706,-0.495023,0.114794,0.146386,-0.224646,1,2
2,0.411764,-0.369546,-0.458490,0.750522,0.272935,0.156402,-0.035687,0.473842,0.258403,0.226648,...,0.014910,-0.335780,-0.261537,-0.055006,-0.535237,0.128619,-0.002265,-0.299280,18,2
3,0.295460,-0.279597,-0.391203,0.591426,0.260882,0.035994,0.024839,0.419069,0.289642,0.286214,...,0.036179,-0.369703,-0.202859,-0.062387,-0.515599,0.133126,0.114751,-0.308144,18,0
4,0.152593,-0.360361,-0.390880,0.682492,0.248925,0.086876,-0.114154,0.464449,0.312763,0.153325,...,0.149875,-0.408616,-0.186768,-0.166792,-0.521301,0.086317,-0.013619,-0.251862,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.243217,-0.364867,-0.139583,0.462050,0.301259,0.010352,-0.179530,0.463264,0.258317,0.404826,...,-0.050681,-0.273107,-0.216582,-0.308223,-0.252769,-0.160325,0.393322,-0.150267,8,0
13180,-0.147072,-0.474946,-0.541926,0.058190,0.418753,0.205028,-0.064300,0.141863,0.383571,0.591876,...,-0.221519,0.413787,-0.232886,0.261513,0.124635,-0.654871,0.059175,0.336033,8,3
13181,-0.064110,-0.434067,-0.363108,-0.106469,0.359295,0.107215,-0.102639,-0.090925,0.412433,0.554757,...,-0.109178,0.276867,-0.224465,-0.073526,-0.002677,-0.719003,0.097201,0.432309,8,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.001068,-0.231221,0.000838,0.249569,0.265003,-0.073580,0.127423,0.242214,0.132900,0.398814,...,-0.072656,0.151674,-0.177812,-0.021540,-0.043838,-0.014226,0.091019,0.105136,8.732630,2.190382
std,0.201528,0.216911,0.213963,0.223258,0.200467,0.224722,0.211291,0.218313,0.203253,0.187996,...,0.188473,0.212186,0.205594,0.180352,0.222938,0.200482,0.218672,0.223542,5.582999,1.315339
min,-0.851681,-1.033672,-0.754178,-0.475520,-0.511140,-0.748549,-0.587477,-0.535000,-0.510817,-0.360000,...,-0.894773,-0.644147,-0.823273,-0.800715,-0.908072,-0.874544,-0.517820,-0.686976,0.000000,0.000000
25%,-0.127246,-0.384455,-0.141505,0.089579,0.126838,-0.237637,-0.017554,0.095262,-0.012553,0.271573,...,-0.191774,0.004943,-0.314840,-0.134339,-0.202913,-0.139861,-0.064561,-0.058702,4.000000,1.000000
50%,-0.000320,-0.227526,-0.006335,0.252039,0.270874,-0.072280,0.124354,0.243663,0.130899,0.397861,...,-0.067503,0.151409,-0.177991,-0.016015,-0.051792,-0.018513,0.077526,0.105406,9.000000,3.000000
75%,0.141735,-0.080188,0.135752,0.408989,0.404457,0.084271,0.280066,0.387163,0.269340,0.531564,...,0.056273,0.298187,-0.038730,0.091700,0.116081,0.117827,0.224924,0.263553,14.000000,3.000000
max,0.622382,0.440326,0.798161,0.918830,0.981927,0.717013,0.837163,1.025694,0.837453,1.053211,...,0.695527,0.873034,0.573492,0.667032,0.627607,0.820333,0.922495,0.902125,19.000000,4.000000


# Community Detection

In [317]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.03589380074751901
  i1 conductance: 0.03589380074751901
=>conductance: 0.03589380074751901
K = 5
  i0 conductance: 0.051339915373765865
  i1 conductance: 0.09157509157509157
  i2 conductance: 0.08670645100315934
  i3 conductance: 0.010940658817933166
  i4 conductance: 0.17074552000323612
=>conductance: 0.010940658817933166
K = 10
  i0 conductance: 0.2669570628500311
  i1 conductance: 0.2791250594388968
  i2 conductance: 0.1119078104993598
  i3 conductance: 0.22189662697175636
  i4 conductance: 0.2321487151448879
  i5 conductance: 0.4796783307976527
  i6 conductance: 0.26710310965630113
  i7 conductance: 0.07347931873479319
  i8 conductance: 0.45339498661474814
  i9 conductance: 0.14634405178817786
=>conductance: 0.07347931873479319
K = 20
  i0 conductance: 0.42911409578076243
  i1 conductance: 0.27916230366492145
  i2 conductance: 0.5235955056179775
  i3 conductance: 0.48232154769846564
  i4 conductance: 0.46770025839793283
  i5 conductance: 0.5736600306278714

In [318]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.42676536769046264
K = 5
NMI:  0.6362632943859701
K = 10
NMI:  0.4983895793604809
K = 20
NMI:  0.30552653684377823


# Node Classification

In [ ]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0,128) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.2f}")
    print(f"  macro_f1: {macro_f1*100:.2f}")

k = 2
  micro_f1: 92.80
  macro_f1: 92.80
k = 5
